In [2]:
import os
os.chdir(os.path.dirname(os.getcwd()))
print(os.getcwd())

from studytimeboard.utils.gsheet import GoogleSheet
from studytimeboard.data_analysis import *
from studytimeboard.constant import *
from studytimeboard import db
from studytimeboard import StudyEventDB

/home/congyu/congyu_program/pythons/inits/p_study_time_board


/home/congyu/.virtualenvs/congyuml/lib/python3.6/site-packages/flask_sqlalchemy/__init__.py:834: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [3]:
gs = GoogleSheet.read_from(STUDY_TIME_TABLE_NAME)

In [4]:
df_dur = gs.sheet(sheet_name=SHEET1, least_col_name=START_TIME)
df_eve = gs.sheet(sheet_name=SHEET2, least_col_name=NAME)

df = merge_dur_eve(df_dur, df_eve)

In [5]:
df.head()

,name,date,start_time,end_time
0,Alpha,2020.11.09,11:00,12:00
1,Delta,2020.11.09,11:00,12:00
2,Delta,2020.11.09,08:00,15:00
3,Beta,2020.11.09,08:00,15:00
4,Diqing,2020.11.09,19:00,23:00


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 284 entries, 0 to 142
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        284 non-null    object
 1   date        284 non-null    object
 2   start_time  284 non-null    object
 3   end_time    284 non-null    object
dtypes: object(4)
memory usage: 11.1+ KB


In [3]:
db.create_all()

In [5]:
def df2studyeventsdb(df):
    study_events = []
    for i, row in df.iterrows():
        study_events.append(StudyEventDB(username=row[NAME],
                                         date=date2datetime(row[DATE]),
                                         start_time=time2datetime(row[START_TIME]),
                                         end_time=time2datetime(row[END_TIME])))
    return study_events


def studyeventsdb2df(db):
    study_events = StudyEventDB.query.all()

    df_dict = {}
    df_dict[NAME] = [se.username for se in study_events]
    df_dict[DATE] = [datetime2date(se.date) for se in study_events]
    df_dict[START_TIME] = [datetime2time(se.start_time) for se in study_events]
    df_dict[END_TIME] = [datetime2time(se.end_time) for se in study_events]

    return pd.DataFrame(df_dict)


In [7]:

gs = GoogleSheet.read_from(STUDY_TIME_TABLE_NAME)
df_dur = gs.sheet(sheet_name=SHEET1, least_col_name=START_TIME)
df_eve = gs.sheet(sheet_name=SHEET2, least_col_name=NAME)

df = merge_dur_eve(df_dur, df_eve)


StudyEventDB.query.delete()
db.session.commit()

# load df to db
study_events = df2studyeventsdb(df)
db.session.add_all(study_events)
db.session.commit()
# for study_event in study_events:
#    db.session.add(study_event)
#    db.session.commit()

df2 = studyeventsdb2df(db)


False


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 284 entries, 0 to 142
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        284 non-null    object
 1   date        284 non-null    object
 2   start_time  284 non-null    object
 3   end_time    284 non-null    object
dtypes: object(4)
memory usage: 11.1+ KB


In [21]:
df = df.reset_index(drop=True)

In [22]:
df2 = df2.reset_index(drop=True)

In [9]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284 entries, 0 to 283
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        284 non-null    object
 1   date        284 non-null    object
 2   start_time  284 non-null    object
 3   end_time    284 non-null    object
dtypes: object(4)
memory usage: 9.0+ KB


In [39]:
import numpy as np
np.array_equal(df.values, df2.values)

print(df.values.shape)
print(df2.values.shape)

print(df.values)
print()
print("-"*50)
print()
print(df2.values)

(284, 4)
(284, 4)
[['Alpha' '2020.11.09' '11:00' '12:00']
 ['Delta' '2020.11.09' '11:00' '12:00']
 ['Delta' '2020.11.09' '08:00' '15:00']
 ...
 ['vivi' '2021.01.05' '13:23' '13:53']
 ['Shangsu' '2021.01.11' '10:07' '10:37']
 ['Congyu' '2021.01.11' '09:15' '09:45']]

--------------------------------------------------

[['Alpha' '2020.11.09' '11:00' '12:00']
 ['Delta' '2020.11.09' '11:00' '12:00']
 ['Delta' '2020.11.09' '08:00' '15:00']
 ...
 ['vivi' '2021.01.05' '13:23' '13:53']
 ['Shangsu' '2021.01.11' '10:07' '10:37']
 ['Congyu' '2021.01.11' '09:15' '09:45']]


In [45]:
for i, j in zip(df.values.ravel(), df2.values.ravel()):
    if i!=j:
        print("{}!={}".format(i, j))
        print(type(i))
        print(type(j))
        print(len(i))
        print(len(j))

8:30!=08:30
<class 'str'>
<class 'str'>
4
5
9:30!=09:30
<class 'str'>
<class 'str'>
4
5


In [11]:
for i, row in df.iterrows():
    print(row["date"])
    print(type(row["start_time"]))
    break

2020.11.09
<class 'str'>
